# Assignment 4: ReAct with Retrieval Tools for Solving Multi-hop Questions

## Meta Instructions
1. Environment:
    - Install the required package using: `pip install sentence_transformers`
    - Additional packages may be required depending on your implementation.

2. Finish coding tasks according to instructions in this file, and change the code in the required area, which is indicated by "Your code starts here" and "Your code ends here".

3. Please summarize all your implementation process, experiment details, results, analysis, discussion and thoughts into a 2-page report. It is encouraged that you include a short proposal for future improvement in the conclusion part. Please submit the report in PDF format.

4. Submission: submit a zip file named "StuID_Name.zip" (e.g., "A0123456J_Wang-Wenjie.zip") to Canvas **Assignments -> Assignment4**. Note that it is **NOT** NUSNET ID. The zip file should **only** include "StuID_Assignment_4.ipynb" and "StuID_Assignment_4.pdf" with your implemented code. The submissison deadline is **23:59 on Mar 28**.

5. Please strictly follow the above instructions, otherwise a grade deduction will be conducted.

### Grading Rules:  
1. Successfully completing the missing parts in the notebook, running the full workflow, and submitting a well-structured report that demonstrates your reasoning will earn you 8/10 of the total score.
2. **(Optional Puzzle)**: If you aim for a higher score and want to earn 1–2 bonus points (without exceeding the maximum score of 60), try incorporating and comparing additional techniques in pre-retrieval (query routing, query rewriting, query expansion), post-retrieval (reranking, summarization, fusion), and indexing (chunking, embedding). Refer to Week 8 slides for guidance. Supplementary materials:
    - [Microsoft](https://learn.microsoft.com/en-us/azure/architecture/ai-ml/guide/rag/rag-solution-design-and-evaluation-guide)
    - [LlamaIndex](https://docs.llamaindex.ai/en/stable/getting_started/concepts/)
    - [LangChain](https://python.langchain.com/docs/introduction/)

### For any questions, please do one of the following actions with priority:
1. Search for similar questions on Slack (https://app.slack.com/client/T088V95D8LC/C088L557RK8).
2. Propose a new question on Slack if not already answered.
3. For non-public questions, e-mail to Xiangyan Liu (e0950125@u.nus.edu) and Pengfei Zhou (e1374451@u.nus.edu) with the subject starting with "CS5260 2025 Spring"

In [3]:
import json
# -----------------------------
# 1. Dataset Loading
# -----------------------------
def load_corpus(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    docs = []
    for entry in data:
        text = entry.get("body", "").strip()
        if text:
            docs.append(text)
    return docs

def load_queries(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        queries = json.load(f)
    return queries[:15]

In [4]:
# -----------------------------
# 2. Retrieval Tool Implementations
# -----------------------------
from rank_bm25 import BM25Okapi
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import torch.nn.functional as F
nltk.download('punkt_tab')
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

def chunk_text(text, chunk_size=50000, overlap=100):
    if len(text) <= chunk_size:
        return [text]
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks


class BM25Retriever:
    def __init__(self, documents):
        """
        BM25Retriever: A traditional lexical retrieval model based on BM25.
        It ranks documents based on term frequency and inverse document frequency (TF-IDF).
        Useful for keyword-based queries.
        """
        ################################
        # Your code starts here
        ################################
        self.documents = [chunk for doc in documents for chunk in chunk_text(doc.lower())]
        tokenized_docs = [word_tokenize(doc) for doc in self.documents]
        self.bm25 = BM25Okapi(tokenized_docs)

        ################################
        # Your code ends here
        ################################

    def retrieve(self, query, top_k=3):
        """
        Retrieves the top_k most relevant documents based on BM25 scoring.
        """
        ################################
        # Your code starts here
        ################################
        tokenized_query = word_tokenize(f"Find documents about: {query.lower()}")
        doc_scores = self.bm25.get_scores(tokenized_query)
        #top_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:top_k]
        top_indices = np.argsort(doc_scores)[::-1][:top_k]
        results = [(self.documents[i], doc_scores[i]) for i in top_indices]

        return results

        ################################
        # Your code ends here
        ################################


class SemanticRetriever:
    def __init__(self, documents, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        """
        SemanticRetriever: Uses transformer-based embeddings to retrieve documents based on semantic similarity.
        Embeds documents using a pre-trained model and ranks them using cosine similarity.
        """
        ################################
        # Your code starts here
        ################################
        self.documents = [chunk for doc in documents for chunk in chunk_text(doc.lower())]
        self.model = SentenceTransformer(model_name)
        self.doc_embeddings = self.model.encode(self.documents, convert_to_tensor=True)


        ################################
        # Your code ends here
        ################################

    def retrieve(self, query, top_k=3):
        """
        Retrieves the top_k most relevant documents using cosine similarity.
        """
        ################################
        # Your code starts here
        ################################
        query_embedding = self.model.encode(f"Find documents about: {query.lower()}", convert_to_tensor=True)
        cos_scores = F.cosine_similarity(query_embedding.unsqueeze(0), self.doc_embeddings, dim=1)
        top_indices = torch.argsort(cos_scores, descending=True)[:top_k].tolist()
        results = [(self.documents[i], cos_scores[i].item()) for i in top_indices]
        return results if results else []

        ################################
        # Your code ends here
        ################################


class HybridRetriever:
    def __init__(self, documents, bm25_weight=0.7, semantic_weight=0.3, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        """
        HybridRetriever: Combines BM25 and Semantic Retrieval.
        Uses a weighted combination of both scores to retrieve documents.
        """
        ################################
        # Your code starts here
        ################################
        self.documents = documents
        self.bm25_weight = bm25_weight
        self.semantic_weight = semantic_weight

        self.bm25_retriever = BM25Retriever(documents)
        self.semantic_retriever = SemanticRetriever(documents, model_name)

        ################################
        # Your code ends here
        ################################

    def retrieve(self, query, top_k=3):
        """
        Retrieves the top_k most relevant documents using a combination of BM25 and semantic scores.
        """
        ################################
        # Your code starts here
        ################################
        bm25_results = self.bm25_retriever.retrieve(query, top_k=len(self.documents))
        semantic_results = self.semantic_retriever.retrieve(query, top_k=len(self.documents))

        combined_scores = {}

        # Normalize BM25 scores
        max_bm25_score = max([score for _, score in bm25_results]) if bm25_results else 1.0
        for doc, score in bm25_results:
            normalized_score = score / max_bm25_score if max_bm25_score > 0 else 0
            combined_scores[doc] = self.bm25_weight * normalized_score

        # Add semantic scores
        for doc, score in semantic_results:
            if doc in combined_scores:
                combined_scores[doc] += self.semantic_weight * score
            else:
                combined_scores[doc] = self.semantic_weight * score

        sorted_results = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
        return sorted_results

        ################################
        # Your code ends here
        ################################

2025-03-28 20:37:29.291398: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt_tab to /Users/Xinyi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
# -----------------------------
# 2. Retrieval Tool Implementations
# -----------------------------
from rank_bm25 import BM25Okapi
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import torch.nn.functional as F


class BM25Retriever:
    def __init__(self, documents):
        """
        BM25Retriever: A traditional lexical retrieval model based on BM25.
        It ranks documents based on term frequency and inverse document frequency (TF-IDF).
        Useful for keyword-based queries.
        """
        ################################
        # Your code starts here
        ################################
        nltk.download('punkt_tab')
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')

        # Tokenize the documents
        self.documents = documents
        tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]
        self.bm25 = BM25Okapi(tokenized_docs)

        ################################
        # Your code ends here
        ################################

    def retrieve(self, query, top_k=3):
        """
        Retrieves the top_k most relevant documents based on BM25 scoring.
        """
        ################################
        # Your code starts here
        ################################
        # Tokenize the query
        tokenized_query = word_tokenize(query.lower())

        # Get document scores
        doc_scores = self.bm25.get_scores(tokenized_query)

        # Get top k documents with their scores
        top_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:top_k]
        results = [(self.documents[i], doc_scores[i]) for i in top_indices]

        return results

        ################################
        # Your code ends here
        ################################


class SemanticRetriever:
    def __init__(self, documents, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        """
        SemanticRetriever: Uses transformer-based embeddings to retrieve documents based on semantic similarity.
        Embeds documents using a pre-trained model and ranks them using cosine similarity.
        """
        ################################
        # Your code starts here
        ################################
        self.documents = documents
        self.model = SentenceTransformer(model_name)

        # Pre-compute document embeddings
        self.doc_embeddings = self.model.encode(documents, convert_to_tensor=True)


        ################################
        # Your code ends here
        ################################

    def retrieve(self, query, top_k=3):
        """
        Retrieves the top_k most relevant documents using cosine similarity.
        """
        ################################
        # Your code starts here
        ################################
        # Encode the query
        query_embedding = self.model.encode(query, convert_to_tensor=True)

        # Calculate cosine similarity
        cos_scores = F.cosine_similarity(query_embedding.unsqueeze(0), self.doc_embeddings, dim=1)

        # Get top k documents with their scores
        top_indices = torch.argsort(cos_scores, descending=True)[:top_k].tolist()
        results = [(self.documents[i], cos_scores[i].item()) for i in top_indices]
        return results if results else []

        ################################
        # Your code ends here
        ################################


class HybridRetriever:
    def __init__(self, documents, bm25_weight=0.5, semantic_weight=0.5, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        """
        HybridRetriever: Combines BM25 and Semantic Retrieval.
        Uses a weighted combination of both scores to retrieve documents.
        """
        ################################
        # Your code starts here
        ################################
        self.documents = documents
        self.bm25_weight = bm25_weight
        self.semantic_weight = semantic_weight

        # Initialize both retrievers
        self.bm25_retriever = BM25Retriever(documents)
        self.semantic_retriever = SemanticRetriever(documents, model_name)

        ################################
        # Your code ends here
        ################################

    def retrieve(self, query, top_k=3):
        """
        Retrieves the top_k most relevant documents using a combination of BM25 and semantic scores.
        """
        ################################
        # Your code starts here
        ################################
        # Get results from both retrievers
        bm25_results = self.bm25_retriever.retrieve(query, top_k=len(self.documents))
        semantic_results = self.semantic_retriever.retrieve(query, top_k=len(self.documents))

        # Create a dictionary to store combined scores
        combined_scores = {}

        # Normalize BM25 scores
        max_bm25_score = max([score for _, score in bm25_results]) if bm25_results else 1.0
        for doc, score in bm25_results:
            normalized_score = score / max_bm25_score if max_bm25_score > 0 else 0
            combined_scores[doc] = self.bm25_weight * normalized_score

        # Add semantic scores
        for doc, score in semantic_results:
            if doc in combined_scores:
                combined_scores[doc] += self.semantic_weight * score
            else:
                combined_scores[doc] = self.semantic_weight * score

        # Sort and return top k results
        sorted_results = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
        return sorted_results

        ################################
        # Your code ends here
        ################################

In [6]:
# -----------------------------
# 3. Generic LLM API Integration
# -----------------------------
from openai import OpenAI
import requests

def query_llm_api(prompt, base_url, api_key):
    """
    Generic LLM API caller using OpenAI client.

    Parameters:
        prompt: A string prompt to send to the LLM.
        base_url: The base URL of the LLM provider's API.
        api_key: Your API key.

    Returns:
        A string response from the LLM.
    """
    client = OpenAI(
        api_key=api_key,
        base_url=base_url
    )

    try:
        response = client.chat.completions.create(
            model="Qwen/Qwen2.5-7B-Instruct",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_completion_tokens=2048
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"API Error: {e}")
        return f"API Error: {str(e)}"



# def query_llm_api(prompt, base_url, api_key):
#     """
#     Generic LLM API caller.

#     Parameters:
#         prompt: A string prompt to send to the LLM.
#         base_url: The base URL of the LLM provider's API.
#         api_key: Your API key.

#     Returns:
#         A string response from the LLM.
#     """
#     headers = {
#         "Content-Type": "application/json",
#         "Authorization": f"Bearer {api_key}"
#     }
#     payload = {
#         "prompt": prompt,
#         "max_tokens": 1024,
#         "model": "Qwen/Qwen2.5-7B-Instruct"
#     }
#     try:
#         response = requests.post(base_url, json=payload, headers=headers, timeout=15)
#         if response.status_code == 200:
#             result = response.json()
#             return result.get("text", "")
#         else:
#             return f"API Error: {response.status_code}, {response.text}"
#     except Exception as e:
#         return f"Exception during API call: {str(e)}"

In [7]:
# -----------------------------
# 5. Evaluation Function with Retrieval Metrics and Exact Match
# -----------------------------
import re

def normalize_text(text):
    text = re.sub(r'\s+', ' ', text)  # Merge whitespace
    return text.lower().strip()

def evaluate_system(queries, retrieval_tool_obj, base_url, api_key):
    """
    Evaluate system performance on a set of queries

    Calculates three key metrics:
    - Recall: Proportion of gold evidence found in retrieval results
    - MRR (Mean Reciprocal Rank): Average reciprocal of the first correct answer's rank
    - EM (Exact Match): Proportion of final answers that exactly match the gold standard
    """
    eval_results = []
    total_recall = 0
    total_mrr = 0
    total_em = 0

    for q in queries:
        # step 1: get predicted answer and retrieved evidence
        pred_answer, retrieved_evidence = react_chain(q["query"], retrieval_tool_obj, base_url, api_key)

        # step 2: prepare data for evaluation
        gold_answer = normalize_text(q["answer"])
        pred_answer = normalize_text(pred_answer)
        gold_evidence = [normalize_text(e["fact"]) for e in q["evidence_list"]]
        retrieved_evidence = [normalize_text(e) for e in retrieved_evidence]

        # step 3: metrics
        ## step 3.1: recall
        found_evidence = 0
        for gold in gold_evidence:
            if any(gold in retrieved for retrieved in retrieved_evidence):
                found_evidence += 1
        recall = found_evidence / len(gold_evidence) if gold_evidence else 0

        ## step 3.2: mrr
        mrr = 0
        for rank, retrieved in enumerate(retrieved_evidence, 1):
            if any(gold in retrieved for gold in gold_evidence):
                mrr = 1 / rank
                break

        ## step 3.3: exact match
        em = 1 if gold_answer == pred_answer else 0

        total_recall += recall
        total_mrr += mrr
        total_em += em

        # step 4: aggregate results
        result = {
            "query": q["query"],
            "gold_answer": gold_answer,
            "pred_answer": pred_answer,
            "recall": recall,
            "mrr": mrr,
            "em": em,
            "retrieved_evidence": retrieved_evidence
        }
        eval_results.append(result)

    num_queries = len(queries)
    overall_metrics = {
        "avg_recall": total_recall / num_queries,
        "avg_mrr": total_mrr / num_queries,
        "avg_em": total_em / num_queries
    }

    return eval_results, overall_metrics

In [8]:
# -----------------------------
# 4. ReAct Reasoning Chain (Max 5 Tool Calls)
# -----------------------------
def react_chain(query, retrieval_tool_obj, base_url, api_key, max_steps=5):
    """
    ReAct reasoning chain that produces a final answer.

    Parameters:
        query: The query string.
        retrieval_tool_obj: An instance of one of the retrieval tool classes.
        base_url: The LLM provider base URL.
        api_key: The API key for the LLM provider.
        max_steps: Maximum number of iterations (tool calls) allowed (max 5).

    Returns:
        pred_answer: a single word or a short phrase
        retrieved_evidence: a list of retrived evidences (list(str))
    """
        ################################
        # Your code starts here
        ################################
    retrieved_evidence = []
    thoughts_history = []

    current_step = 0
    final_answer = None

    initial_prompt = f"""
        You are an expert for solving a multi-hop question answering task using a ReAct framework.
        The question requires retrieving multiple pieces of information and reasoning over them.

        QUESTION: {query}

        Let's solve this step by step. At each step, you can:
        1. THINK: Reason about what information you need next.
        2. ACTION: Retrieve information by reformulating a search query.
        3. OBSERVE: Review the retrieved information.
        4. ANSWER: When you have enough information, provide a final answer.

        Start by thinking about what information you need.
        """

    while current_step < max_steps and final_answer is None:
        current_step += 1
        chat_history = "\n\n".join(thoughts_history)
        prompt = initial_prompt + "\n\n" + chat_history

        if current_step > 1:
            prompt += "\n\nWhat's your next step?"

        llm_response = query_llm_api(prompt, base_url, api_key)

        if "ACTION:" in llm_response:
            search_query = llm_response.split("ACTION:")[1].strip()
            if "OBSERVE:" in search_query:
                search_query = search_query.split("OBSERVE:")[0].strip()

            # Use the retrieval tool to get relevant documents
            retrieved_docs = retrieval_tool_obj.retrieve(search_query, top_k=2)

            retrieved_text = "\n\n".join([doc for doc, _ in retrieved_docs])

            retrieved_evidence.append(retrieved_text)

            thoughts_history.append(f"THINK: {llm_response.split('ACTION:')[0].strip()}")
            thoughts_history.append(f"ACTION: {search_query}")
            thoughts_history.append(f"OBSERVE: {retrieved_text}")

        elif "ANSWER:" in llm_response:
            answer_text = llm_response.split("ANSWER:")[1].strip()
            final_answer = answer_text

            if "THINK:" in llm_response:
                thoughts_history.append(f"THINK: {llm_response.split('THINK:')[1].split('ANSWER:')[0].strip()}")
            thoughts_history.append(f"ANSWER: {answer_text}")

        else:
            thoughts_history.append(f"THINK: {llm_response}")

            #follow-up prompt for ReAct format
            follow_up = """
                Please follow this ReAct format:
                1. THINK: Analyze what information you have and what you need.
                2. ACTION: Formulate a specific search query to retrieve information.
                OR
                1. THINK: Analyze all the information you've gathered.
                2. ANSWER: Provide your final answer to the original question.
                """
            llm_response = query_llm_api(follow_up, base_url, api_key)

    # if max
    if final_answer is None:
        final_prompt = f"""
            Based on all the information gathered so far:

            {query}

            Retrieved evidence:
            {' '.join(retrieved_evidence)}

            What is the single answer to the original question? Respond with just the entity name (person, place, organization, etc.) that answers the question.
            """
        final_answer = query_llm_api(final_prompt, base_url, api_key)

    if final_answer and len(final_answer.split()) > 10:
        final_answer = " ".join(final_answer.split()[:10])

        ################################
        # Your code ends here
        ################################
    return final_answer, retrieved_evidence

In [ ]:
# Config
base_url = "https://api.deepinfra.com/v1/openai"
api_key = "JzKUj7JtqFsyrzAIdCJkXu5T6iSQtvwC"

# Load dataset
documents = load_corpus("corpus.json")
queries = load_queries("MultiHopRAG.json")


retriever = "bm25" # choices ["bm25", "semantic", "hybrid"]
if retriever == "bm25":
    retrieval_tool = BM25Retriever(documents)
elif retriever == "semantic":
    retrieval_tool = SemanticRetriever(documents)
elif retriever == "hybrid":
    retrieval_tool = HybridRetriever(documents)
else:
    raise ValueError("Unsupported retriever type.")

# Evaluate the system on all queries using the chosen retrieval tool
eval_results, overall_metrics = evaluate_system(queries, retrieval_tool, base_url, api_key)

